In [1]:
from IPython.display import display
import os

# 上傳檔案（會跳出檔案選擇器）
from ipywidgets import FileUpload

upload = FileUpload()
display(upload)


FileUpload(value={}, description='Upload')

In [2]:
import os
from pathlib import Path

# 假設你只上傳了一個檔案
# Check if any file is uploaded and get the filename
if upload.value:
    # Get the first (and likely only) filename from the dictionary keys
    filename = list(upload.value.keys())[0]
    fileinfo = upload.value[filename] # Access the file info dictionary using the filename as key
else:
    print("No file uploaded.")
    # Handle the case where no file is uploaded, perhaps by exiting or prompting the user.
    # For this example, we'll assume a file was uploaded as per the traceback context.
    raise FileNotFoundError("No file was uploaded.")


In [3]:
# 顯示內容結構（除錯用）
print(fileinfo)

# 儲存 kaggle.json
# filename is already obtained above
content = fileinfo['content']

kaggle_dir = Path.home() / ".kaggle"
kaggle_dir.mkdir(exist_ok=True)

kaggle_json_path = kaggle_dir / "kaggle.json"
with open(kaggle_json_path, "wb") as f:
    f.write(content)

{'metadata': {'name': 'kaggle.json', 'type': 'application/json', 'size': 64, 'lastModified': 1745215961986}, 'content': b'{"username":"suchiwen","key":"01a925cee9e9e9d232008524b0434fb9"}'}


In [4]:
# 設定權限（Linux/macOS 建議）
os.chmod(kaggle_json_path, 0o600)

print(f"{filename} 已成功儲存至 {kaggle_json_path}")

!kaggle datasets list -s cifar

!pip install -U kaggle
!pip install --upgrade pandas
import os
import zipfile

kaggle.json 已成功儲存至 /root/.kaggle/kaggle.json
ref                                                     title                                                  size  lastUpdated                 downloadCount  voteCount  usabilityRating  
------------------------------------------------------  ----------------------------------------------  -----------  --------------------------  -------------  ---------  ---------------  
fedesoriano/cifar100                                    CIFAR-100 Python                                  168517809  2020-12-26 08:37:10.143000          12486        178  1.0              
pankrzysiu/cifar10-python                               CIFAR-10 Python                                   340613496  2018-01-27 13:42:40.967000          15047        255  0.75             
petitbonney/cifar10-image-recognition                   CIFAR-10                                         1007971063  2019-10-01 12:50:23.227000           2964         27  0.8235294        
valentynsi

In [5]:
# 建立 Kaggle 資料夾
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 下載 Dog Breed Identification 資料集
!kaggle competitions download -c dog-breed-identification --force
!unzip -oq dog-breed-identification.zip -d dog-breed-identification

cp: cannot stat 'kaggle.json': No such file or directory
 88% 608M/691M [00:00<00:00, 1.19GB/s]
100% 691M/691M [00:00<00:00, 778MB/s] 


In [23]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [24]:
# --- 定義模型 ---
class MultiInputCNN(nn.Module):
    def __init__(self, num_classes=120):
        super(MultiInputCNN, self).__init__()

        def feature_extractor():
            return nn.Sequential(
                nn.Conv2d(3, 64, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.Conv2d(64, 64, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(2),
                nn.BatchNorm2d(64),

                nn.Conv2d(64, 128, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.Conv2d(128, 128, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(2),
                nn.BatchNorm2d(128),

                nn.Flatten()
            )

        self.ear_branch = feature_extractor()
        self.tail_branch = feature_extractor()
        self.nose_branch = feature_extractor()

        # 計算flatten後的大小，假設輸入尺寸64x64
        # 經兩次 MaxPool2d(2) -> size: 64x64 -> 16x16
        # 最後channel為128
        flattened_size = 128 * 16 * 16 * 3  # 三個 branch 串接

        self.fc = nn.Sequential(
            nn.Linear(flattened_size, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, ear, tail, nose):
        x1 = self.ear_branch(ear)
        x2 = self.tail_branch(tail)
        x3 = self.nose_branch(nose)
        x = torch.cat((x1, x2, x3), dim=1)
        return self.fc(x)

In [25]:
# --- 自訂 Dataset ---
class DogDataset(Dataset):
    def __init__(self, ears, tails, noses, labels):
        self.ears = ears
        self.tails = tails
        self.noses = noses
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return (self.ears[idx], self.tails[idx], self.noses[idx]), self.labels[idx]


In [13]:
# --- 測試資料 Dataset ---
class TestDogDataset(Dataset):
    def __init__(self, ears, tails, noses):
        self.ears = ears
        self.tails = tails
        self.noses = noses

    def __len__(self):
        return len(self.ears)

    def __getitem__(self, idx):
        return self.ears[idx], self.tails[idx], self.noses[idx]


Preparing data for PyTorch... This is a placeholder function.
You need to implement the logic to load images, extract ear, tail, nose regions, preprocess, and save as .npy files.
Looking for images in: dog-breed-identification/train
Looking for labels in: dog-breed-identification/labels.csv
Dummy data created. Replace with real data processing.


In [26]:
# 訓練設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultiInputCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [29]:
# --- 資料準備函數（需自行替換為實際圖像處理） ---
def prepare_pytorch_data(image_dir, labels_path, image_size=(64, 64)):
    print("⚠️ 你必須自行實作此函數以萃取耳朵、尾巴、鼻子圖像區塊。")
    print(f"暫時使用虛擬資料作為示範，目標圖片尺寸：{image_size}")

    num_samples = 1000  # 假設有1000筆訓練資料
    dummy_shape = (num_samples, image_size[0], image_size[1], 3)

    X_ear_dummy = np.random.rand(*dummy_shape).astype(np.float32)
    X_tail_dummy = np.random.rand(*dummy_shape).astype(np.float32)
    X_nose_dummy = np.random.rand(*dummy_shape).astype(np.float32)
    y_dummy = np.random.randint(0, 120, num_samples).astype(np.int64)

    np.save('X_ear.npy', X_ear_dummy)
    np.save('X_tail.npy', X_tail_dummy)
    np.save('X_nose.npy', X_nose_dummy)
    np.save('labels.npy', y_dummy)
def prepare_data_with_local(test_image_dir, test_df, image_size):
    print("⚠️ 你必須自行實作此函數以萃取測試集耳朵、尾巴、鼻子圖像區塊。")
    num_test_samples = len(test_df)
    dummy_shape = (num_test_samples, image_size[0], image_size[1], 3)

    X_test_ear_dummy = np.random.rand(*dummy_shape).astype(np.float32)
    X_test_nose_dummy = np.random.rand(*dummy_shape).astype(np.float32)
    X_test_tail_dummy = np.random.rand(*dummy_shape).astype(np.float32)

    return X_test_ear_dummy, X_test_nose_dummy, X_test_tail_dummy

In [33]:
# --- 資料準備函數（需自行替換為實際圖像處理） ---
def prepare_pytorch_data(image_dir, labels_path, image_size=(64, 64)):
    print("⚠️ 你必須自行實作此函數以萃取耳朵、尾巴、鼻子圖像區塊。")
    print(f"暫時使用虛擬資料作為示範，目標圖片尺寸：{image_size}")

    num_samples = 1000  # 假設有1000筆訓練資料
    dummy_shape = (num_samples, image_size[0], image_size[1], 3)

    X_ear_dummy = np.random.rand(*dummy_shape).astype(np.float32)
    X_tail_dummy = np.random.rand(*dummy_shape).astype(np.float32)
    X_nose_dummy = np.random.rand(*dummy_shape).astype(np.float32)
    y_dummy = np.random.randint(0, 120, num_samples).astype(np.int64)

    np.save('X_ear.npy', X_ear_dummy)
    np.save('X_tail.npy', X_tail_dummy)
    np.save('X_nose.npy', X_nose_dummy)
    np.save('labels.npy', y_dummy)

def prepare_data_with_local(test_image_dir, test_df, image_size):
    print("⚠️ 你必須自行實作此函數以萃取測試集耳朵、尾巴、鼻子圖像區塊。")
    num_test_samples = len(test_df)
    dummy_shape = (num_test_samples, image_size[0], image_size[1], 3)

    X_test_ear_dummy = np.random.rand(*dummy_shape).astype(np.float32)
    X_test_nose_dummy = np.random.rand(*dummy_shape).astype(np.float32)
    X_test_tail_dummy = np.random.rand(*dummy_shape).astype(np.float32)

    return X_test_ear_dummy, X_test_nose_dummy, X_test_tail_dummy

# --- 主要執行區域 ---
if __name__ == "__main__":
    pytorch_image_size = (64, 64)
    download_dir = 'dog-breed-identification'
    train_image_dir = os.path.join(download_dir, 'train')
    labels_file = os.path.join(download_dir, 'labels.csv')

    # 準備訓練資料
    prepare_pytorch_data(train_image_dir, labels_file, image_size=pytorch_image_size)

    # 載入資料並轉換 tensor (N,H,W,C) -> (N,C,H,W)
    X_ear = torch.tensor(np.load('X_ear.npy'), dtype=torch.float32).permute(0,3,1,2)
    X_tail = torch.tensor(np.load('X_tail.npy'), dtype=torch.float32).permute(0,3,1,2)
    X_nose = torch.tensor(np.load('X_nose.npy'), dtype=torch.float32).permute(0,3,1,2)
    y = torch.tensor(np.load('labels.npy'), dtype=torch.long)

    # 分割訓練與驗證
    X_ear_train, X_ear_val, X_tail_train, X_tail_val, X_nose_train, X_nose_val, y_train, y_val = train_test_split(
        X_ear, X_tail, X_nose, y, test_size=0.2, random_state=42
    )

    train_dataset = DogDataset(X_ear_train, X_tail_train, X_nose_train, y_train)
    val_dataset = DogDataset(X_ear_val, X_tail_val, X_nose_val, y_val)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)

    # 設定裝置與模型
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = MultiInputCNN().to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=5e-4, weight_decay=1e-4)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

    # 訓練迴圈
    best_val_loss = float('inf')
    patience = 10
    counter = 0

    for epoch in range(50):
        model.train()
        train_loss = 0.0
        train_correct = 0

        for (ear, tail, nose), labels in train_loader:
            ear, tail, nose, labels = ear.to(device), tail.to(device), nose.to(device), labels.to(device)
            outputs = model(ear, tail, nose)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * labels.size(0)
            train_correct += (outputs.argmax(1) == labels).sum().item()

        model.eval()
        val_loss = 0.0
        val_correct = 0

        with torch.no_grad():
            for (ear, tail, nose), labels in val_loader:
                ear, tail, nose, labels = ear.to(device), tail.to(device), nose.to(device), labels.to(device)
                outputs = model(ear, tail, nose)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * labels.size(0)
                val_correct += (outputs.argmax(1) == labels).sum().item()

        train_loss /= len(train_dataset)
        val_loss /= len(val_dataset)
        train_acc = train_correct / len(train_dataset)
        val_acc = val_correct / len(val_dataset)

        print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Acc={train_acc:.4f} | Val Loss={val_loss:.4f}, Acc={val_acc:.4f}")

        scheduler.step(val_loss)

        # 早停
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping triggered.")
                break

    # 預測階段
    test_df = pd.read_csv(os.path.join(download_dir, 'sample_submission.csv'))

    X_test_ear_np, X_test_nose_np, X_test_tail_np = prepare_data_with_local(
        os.path.join(download_dir, 'test'), test_df, pytorch_image_size
    )

    X_test_ear_tensor = torch.tensor(X_test_ear_np, dtype=torch.float32).permute(0, 3, 1, 2)
    X_test_nose_tensor = torch.tensor(X_test_nose_np, dtype=torch.float32).permute(0, 3, 1, 2)
    X_test_tail_tensor = torch.tensor(X_test_tail_np, dtype=torch.float32).permute(0, 3, 1, 2)

    test_dataset = TestDogDataset(X_test_ear_tensor, X_test_tail_tensor, X_test_nose_tensor)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    model.eval()
    all_preds_probs = []

    with torch.no_grad():
        for ear, tail, nose in test_loader:
            ear, tail, nose = ear.to(device), tail.to(device), nose.to(device)
            outputs = model(ear, tail, nose)
            probs = torch.softmax(outputs, dim=1).cpu().numpy()
            all_preds_probs.append(probs)

    preds_probs = np.concatenate(all_preds_probs, axis=0)
    class_names = list(test_df.columns[1:])  # 除去id欄位

    submission = pd.DataFrame(preds_probs, columns=class_names)
    submission.insert(0, 'id', test_df['id'])
    submission.to_csv('submission_multi_input_cnn.csv', index=False)

    print("✅ Submission file created: submission_multi_input_cnn.csv")


⚠️ 你必須自行實作此函數以萃取耳朵、尾巴、鼻子圖像區塊。
暫時使用虛擬資料作為示範，目標圖片尺寸：(64, 64)
Epoch 1: Train Loss=4.8796, Acc=0.0037 | Val Loss=4.7948, Acc=0.0100
Epoch 2: Train Loss=3.0262, Acc=0.3125 | Val Loss=4.8458, Acc=0.0100
Epoch 3: Train Loss=1.3592, Acc=0.6700 | Val Loss=4.9387, Acc=0.0050
Epoch 4: Train Loss=0.5972, Acc=0.8500 | Val Loss=4.9782, Acc=0.0050
Epoch 5: Train Loss=0.2949, Acc=0.9237 | Val Loss=4.8903, Acc=0.0050
Epoch 6: Train Loss=0.2218, Acc=0.9437 | Val Loss=4.8434, Acc=0.0100
Epoch 7: Train Loss=0.1599, Acc=0.9550 | Val Loss=4.8418, Acc=0.0100
Epoch 8: Train Loss=0.1178, Acc=0.9688 | Val Loss=4.8666, Acc=0.0100
Epoch 9: Train Loss=0.1109, Acc=0.9637 | Val Loss=4.8948, Acc=0.0050
Epoch 10: Train Loss=0.0710, Acc=0.9862 | Val Loss=4.8912, Acc=0.0050
Epoch 11: Train Loss=0.0814, Acc=0.9775 | Val Loss=4.8823, Acc=0.0100
Early stopping triggered.
⚠️ 你必須自行實作此函數以萃取測試集耳朵、尾巴、鼻子圖像區塊。
✅ Submission file created: submission_multi_input_cnn.csv


In [32]:
  # 預測階段
    test_df = pd.read_csv(os.path.join(download_dir, 'sample_submission.csv'))

    X_test_ear_np, X_test_nose_np, X_test_tail_np = prepare_data_with_local(
        os.path.join(download_dir, 'test'), test_df, pytorch_image_size
    )

    X_test_ear_tensor = torch.tensor(X_test_ear_np, dtype=torch.float32).permute(0, 3, 1, 2)
    X_test_nose_tensor = torch.tensor(X_test_nose_np, dtype=torch.float32).permute(0, 3, 1, 2)
    X_test_tail_tensor = torch.tensor(X_test_tail_np, dtype=torch.float32).permute(0, 3, 1, 2)

    test_dataset = TestDogDataset(X_test_ear_tensor, X_test_tail_tensor, X_test_nose_tensor)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    model.eval()
    all_preds_probs = []

    with torch.no_grad():
        for ear, tail, nose in test_loader:
            ear, tail, nose = ear.to(device), tail.to(device), nose.to(device)
            outputs = model(ear, tail, nose)
            probs = torch.softmax(outputs, dim=1).cpu().numpy()
            all_preds_probs.append(probs)

    preds_probs = np.concatenate(all_preds_probs, axis=0)
    class_names = list(test_df.columns[1:])  # 除去id欄位


IndentationError: unexpected indent (<ipython-input-32-e736a7fa61e5>, line 2)

In [22]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

# 讀取 sample_submission.csv 作為格式參考
test_df = pd.read_csv('dog-breed-identification/sample_submission.csv')

# --- 假設函數：準備耳朵、尾巴、鼻子的測試圖像 ---
def prepare_data_with_local(test_image_dir, test_df, image_size):
    """
    虛擬實作：準備測試圖片的特徵（耳、鼻、尾）區塊。請自行更換為實際的圖像處理邏輯。
    """
    print("✅ Preparing test data... (Placeholder version)")
    num_test_samples = len(test_df)
    dummy_shape = (num_test_samples, image_size[0], image_size[1], 3)
    X_test_ear_dummy = np.random.rand(*dummy_shape).astype(np.float32)
    X_test_nose_dummy = np.random.rand(*dummy_shape).astype(np.float32)
    X_test_tail_dummy = np.random.rand(*dummy_shape).astype(np.float32)
    return X_test_ear_dummy, X_test_nose_dummy, X_test_tail_dummy

# 假設圖像大小定義如下（與訓練一致）
pytorch_image_size = (64, 64)

# 呼叫資料準備函數
X_test_ear_np, X_test_nose_np, X_test_tail_np = prepare_data_with_local(
    'dog-breed-identification/test', test_df, pytorch_image_size
)

# 將 numpy 陣列轉為 torch tensor 並調整為 [B, C, H, W] 格式
X_test_ear_tensor = torch.tensor(X_test_ear_np, dtype=torch.float32).permute(0, 3, 1, 2)
X_test_nose_tensor = torch.tensor(X_test_nose_np, dtype=torch.float32).permute(0, 3, 1, 2)
X_test_tail_tensor = torch.tensor(X_test_tail_np, dtype=torch.float32).permute(0, 3, 1, 2)

# --- 定義 Dataset ---
class TestDogDataset(Dataset):
    def __init__(self, ears, tails, noses):
        self.ears = ears
        self.tails = tails
        self.noses = noses

    def __len__(self):
        return len(self.ears)

    def __getitem__(self, idx):
        return self.ears[idx], self.tails[idx], self.noses[idx]

# 建立 DataLoader
test_dataset = TestDogDataset(X_test_ear_tensor, X_test_tail_tensor, X_test_nose_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# --- 預測階段 ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

all_preds_probs = []

with torch.no_grad():
    for ear, tail, nose in test_loader:
        ear, tail, nose = ear.to(device), tail.to(device), nose.to(device)
        outputs = model(ear, tail, nose)
        probs = torch.softmax(outputs, dim=1).cpu().numpy()
        all_preds_probs.append(probs)

# 合併所有 batch 預測
preds_probs = np.concatenate(all_preds_probs, axis=0)

# ✅ 從 sample_submission 取得正確的品種欄位名稱
class_names = list(test_df.columns[1:])  # 忽略 'id'

# 構建提交 DataFrame
submission = pd.DataFrame(preds_probs, columns=class_names)
submission.insert(0, 'id', test_df['id'])  # 插入 id 欄位

# 儲存為 CSV 檔案
submission.to_csv('submission_multi_input_cnn.csv', index=False)

print("✅ Submission file created: submission_multi_input_cnn.csv")


✅ Preparing test data... (Placeholder version)
✅ Submission file created: submission_multi_input_cnn.csv
